In [1]:
import pandas as pd
import sys,os
from tqdm import tqdm
sys.path.insert(0, '/home/shenwanxiang/Research/aliyun_sync/COMPASS/')
from baseline.immnue_score.scorer import ssGSEA

from compass.utils import plot_embed_with_label
from compass import PreTrainer, FineTuner, loadcompass #, get_minmal_epoch
from compass.utils import plot_embed_with_label,plot_performance, score2
from compass.tokenizer import CANCER_CODE, CONCEPT

/home/shenwanxiang/anaconda3/envs/compass/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
data_path = '/home/shenwanxiang/Research/aliyun_sync/COMPASS/paper/00_data/'
df_label = pd.read_pickle(os.path.join(data_path, 'ITRP.PATIENT.TABLE.ALIGN'))
df_tpm = pd.read_pickle(os.path.join(data_path, 'ITRP.TPM.TABLE'))
df_tpm.shape, df_label.shape

dfcx = df_label.cancer_type.map(CANCER_CODE).to_frame('cancer_code').join(df_tpm)

In [8]:
df_label.to_csv('./data/ITRP_clinical.csv')

In [ ]:
gmt29  = pd.read_excel('./Fges_29_gmt.xlsx',sheet_name= 'Gene Description',skiprows=1)
gmt29 = gmt29.groupby(['Gene signature']).Gene.apply(lambda x:x.tolist())

res=[]
for name, g_list in tqdm(gmt29.items()):
    gsea = ssGSEA(g_list, name)
    score = gsea.fit_transform(df_tpm)
    res.append(score) 

dfc = pd.concat(res,axis=1)
dfc.columns = dfc.columns.map(lambda x:x.replace('NES_', ''))
dfc.loc[df_label.index].to_csv('./data/ITRP_ssGSEA_Fegs29.csv')

In [41]:
def _connect(x):
    from itertools import chain
    return list(set(chain(*[i.split(':') for i in x])))
    
concept_genes = CONCEPT.groupby('BroadCelltypePathway').Genes.apply(lambda x:x.tolist()).apply(_connect)

res=[]
for name, g_list in tqdm(concept_genes.items()):
    gsea = ssGSEA(g_list, name)
    score = gsea.fit_transform(df_tpm)
    res.append(score) 

dfc = pd.concat(res,axis=1)
dfc.columns = dfc.columns.map(lambda x:x.replace('NES_', ''))
dfc = dfc[CONCEPT.groupby('BroadCelltypePathway').Concept_index.mean().sort_values().index]
dfc.loc[df_label.index].to_csv('./data/ITRP_ssGSEA_concept43.csv')

43it [06:22,  8.88s/it]


In [52]:
pretrainer = loadcompass('../../../checkpoint/latest/pretrainer.pt', map_location = 'cpu')
dfg, dfc = pretrainer.extract(dfcx,  batch_size= 128)

100%|#####################################################################################| 9/9 [00:28<00:00,  3.21s/it]


In [54]:
dfc.to_csv('./data/ITRP_COMPASS_concept44.csv')